In [45]:
import json
import pandas as pd
import numpy as np
import os
import csv

#maybe doc import
with open("E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json", "r", encoding="utf-8") as file:
    data = json.load(file)

blueTimestamps = {}
redTimestamps = {}
participantData = []
participantsData = data['metadata']['participants']

In [46]:
%run fetchTurretInfo.ipynb
%run sectionFunctions.ipynb

In [47]:
def findClosestCaseNumber(blueTimestamps, redTimestamps, participantId, eventTimestamp):
    # Determine which timestamps dictionary to use based on participantId
    timestamps = blueTimestamps if participantId > 0 and participantId < 6 else redTimestamps
    
    closestCaseNumber = None
    smallestDifference = float('inf')
    
    for caseNumber, timestamp in timestamps.items():
        difference = abs(timestamp - eventTimestamp)
        if difference < smallestDifference:
            smallestDifference = difference
            closestCaseNumber = caseNumber
    
    return closestCaseNumber

In [48]:
def executeClosestCaseFunction(blueTimestamps, redTimestamps, participantId, eventTimestamp, x, y):
    # Use the modified findClosestCaseNumber function
    closestCaseNumber = findClosestCaseNumber(blueTimestamps, redTimestamps, participantId, eventTimestamp)
    
    # Determine the function name prefix based on participantId
    functionPrefix = "blueCase" if participantId > 0 and participantId < 6 else "redCase"
    
    # Construct the function name
    functionName = f"{functionPrefix}{closestCaseNumber}"
    
    # Retrieve the function object using globals() or locals() if the function is defined in a local scope
    functionToCall = globals().get(functionName)
    
    # Check if the function exists
    if functionToCall:
        # Print the name of the function being called
        print(f"Calling function: {functionName}")
        
        # Call the function with x and y, and return its result
        return functionToCall(x, y)
    else:
        return "Function does not exist."

In [49]:
def getTowerDestroyedData(data, participantData):
    towerData = []
    caseBlue = 0
    caseRed = 0
    resetTowerData()
    # Initialize boolean variables for blue team
    bto = bti = bmo = bmi = bbo = bbi = True

    # Initialize boolean variables for red team
    rto = rti = rmo = rmi = rbo = rbi = True
    
    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', None)
                laneType = event['laneType']
                timestamp = event['timestamp']
                
                # Update boolean variables based on tower destruction events
                if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
                    if teamId == 100:
                        if towerType == 'OUTER_TURRET':
                            if laneType == 'TOP_LANE':
                                bto = False  # blue top outer destroyed
                            elif laneType == 'MID_LANE':
                                bmo = False  # blue mid outer destroyed
                            elif laneType == 'BOT_LANE':
                                bbo = False  # blue bot outer destroyed
                        elif towerType == 'INNER_TURRET':
                            if laneType == 'TOP_LANE':
                                bti = False  # blue top inner destroyed
                            elif laneType == 'MID_LANE':
                                bmi = False  # blue mid inner destroyed
                            elif laneType == 'BOT_LANE':
                                bbi = False  # blue bot inner destroyed
                        # Call blueDefCaseCalc with updated boolean variables
                        caseBlue = blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi)
                        # Update blue team timestamps
                        updateBlueTowerData(caseBlue, timestamp)
                    elif teamId == 200:
                        if towerType == 'OUTER_TURRET':
                            if laneType == 'TOP_LANE':
                                rto = False  # red top outer destroyed
                            elif laneType == 'MID_LANE':
                                rmo = False  # red mid outer destroyed
                            elif laneType == 'BOT_LANE':
                                rbo = False  # red bot outer destroyed
                        elif towerType == 'INNER_TURRET':
                            if laneType == 'TOP_LANE':
                                rti = False  # red top inner destroyed
                            elif laneType == 'MID_LANE':
                                rmi = False  # red mid inner destroyed
                            elif laneType == 'BOT_LANE':
                                rbi = False  # red bot inner destroyed
                        # Call redDefCaseCalc with updated boolean variables
                        caseRed = redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi)
                        # Update red team timestamps
                        updateRedTowerData(caseRed, timestamp)

                # Append tower data to the list
                towerData.append({
                    'teamId': teamId,
                    'towerType': towerType,
                    'laneType': laneType,
                    'timestamp': timestamp
                })

    return towerData

# Example usage:
towerData = getTowerDestroyedData(data, participantData)

# Display the extracted data
for tower in towerData:
    teamId = tower['teamId']
    towerType = tower['towerType']
    laneType = tower['laneType']
    timestamp = tower['timestamp']

    if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
        print(f"Team: {teamId}, Tower Type: {towerType}, Lane: {laneType}, timestamp {timestamp}")
        
print("\nBlue Team Timestamps:")
for caseNumber, timestamp in blueTimestamps.items():
    print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")

# Print contents of redTimestamps dictionary
print("\nRed Team Timestamps:")
for caseNumber, timestamp in redTimestamps.items():
    print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")

Team: 100, Tower Type: OUTER_TURRET, Lane: TOP_LANE, timestamp 1074561
Team: 200, Tower Type: OUTER_TURRET, Lane: BOT_LANE, timestamp 1162112
Team: 100, Tower Type: OUTER_TURRET, Lane: BOT_LANE, timestamp 1186441
Team: 100, Tower Type: OUTER_TURRET, Lane: MID_LANE, timestamp 1252821
Team: 200, Tower Type: OUTER_TURRET, Lane: TOP_LANE, timestamp 1349309
Team: 100, Tower Type: INNER_TURRET, Lane: MID_LANE, timestamp 1380698
Team: 200, Tower Type: INNER_TURRET, Lane: BOT_LANE, timestamp 1545648
Team: 100, Tower Type: INNER_TURRET, Lane: BOT_LANE, timestamp 1599129

Blue Team Timestamps:
Case Number: 1, Timestamp: 0
Case Number: 5, Timestamp: 1074561
Case Number: 11, Timestamp: 1186441
Case Number: 12, Timestamp: 1252821
Case Number: 19, Timestamp: 1380698
Case Number: 26, Timestamp: 1599129

Red Team Timestamps:
Case Number: 1, Timestamp: 0
Case Number: 7, Timestamp: 1162112
Case Number: 11, Timestamp: 1349309
Case Number: 16, Timestamp: 1545648


In [50]:
def processParticipantFramesToCsv(filePath, outputCsvPath, blueTimestamps, redTimestamps):
    # Load JSON data
    with open(filePath, 'r') as file:
        data = json.load(file)

    # Define CSV headers
    headers = ['frameNumber', 'participantId', 'x', 'y', 'totalGold', 'totalDamageToChampions', 'safetyStatus']

    # Prepare for CSV writing
    os.makedirs(os.path.dirname(outputCsvPath), exist_ok=True)
    with open(outputCsvPath, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()

        # Iterate through frames
        for frameIndex, frame in enumerate(data['info']['frames']):
            timestamp = frame['timestamp']  # Frame's timestamp
            for participantId, details in frame['participantFrames'].items():
                position = details.get('position', {'x': 0, 'y': 0})
                totalGold = details.get('totalGold', 0)
                # Assuming 'totalDamageToChampions' is available in the participantFrames details
                totalDamageToChampions = details.get('totalDamageToChampions', 0)

                # Calculate safety status for each participant in this frame
                safetyStatus = executeClosestCaseFunction(blueTimestamps, redTimestamps, int(participantId), timestamp, position['x'], position['y'])

                writer.writerow({
                    'frameNumber': frameIndex + 1,
                    'participantId': participantId,
                    'x': position['x'],
                    'y': position['y'],
                    'totalGold': totalGold,
                    'totalDamageToChampions': totalDamageToChampions,
                    'safetyStatus': safetyStatus
                })

In [51]:
inputFilePath = "E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json"
outputCsvPath = "E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json"
processParticipantFramesToCsv(inputFilePath, outputCsvPath, blueTimestamps, redTimestamps)

Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: redCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: blueCase1
Calling function: redCase1
Calling 

In [52]:
file_path = "E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json"
df = pd.read_csv(file_path)

# Filter the DataFrame to keep only rows where the participant ID is 3 or 8
# Adjust 'participant_id' to the actual column name in your CSV file that contains the participant IDs
filtered_df = df[df['participantId'].isin([3, 8])]

# Save the filtered DataFrame to a new CSV file
filtered_file_path = "E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json"
filtered_df.to_csv(filtered_file_path, index=False)

print('Filtered CSV file has been saved.')

Filtered CSV file has been saved.


In [53]:

file_path = "E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json"
df = pd.read_csv(file_path)

# Calculate the progressive average threat
# This groups the DataFrame by 'participantId', then applies the expanding mean on 'safetyStatus' within each group
df['progressive_average_threat'] = df.groupby('participantId')['safetyStatus'].expanding().mean().reset_index(level=0, drop=True)

# Save the modified DataFrame back to a CSV file
output_path ="E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json"
df.to_csv(output_path, index=False)

print(f"Modified file saved to {output_path}")


Modified file saved to E:\DistanceTesting\filenamesv2\EUW1_6871911174.json


In [54]:
df = pd.read_csv(file_path)
df.tail(10)

,frameNumber,participantId,x,y,totalGold,totalDamageToChampions,safetyStatus,progressive_average_threat
48,25,3,5495,7048,6748,0,100,28.000000
49,25,8,7098,7501,11554,0,100,56.000000
50,26,3,4728,5062,6870,0,0,26.923077
51,26,8,12630,14011,12152,0,0,53.846154
52,27,3,605,898,7324,0,0,25.925926
53,27,8,1174,4800,12812,0,100,55.555556
54,28,3,2381,1654,7683,0,0,25.000000
55,28,8,7754,3422,13259,0,100,57.142857
56,29,3,5464,3380,7820,0,100,27.586207
57,29,8,578,2636,14166,0,100,58.620690


In [57]:
file_path='E:\\DistanceTesting\\filenamesv2\\EUW1_6871911174.json'
blueTimestamps = {}
redTimestamps = {}
participantData = []

def demo_one_button_run(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    
   
    towerData = getTowerDestroyedData(data, participantData)

# Display the extracted data
    for tower in towerData:
        teamId = tower['teamId']
        towerType = tower['towerType']
        laneType = tower['laneType']
        timestamp = tower['timestamp']

        if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
            print(f"Team: {teamId}, Tower Type: {towerType}, Lane: {laneType}, timestamp {timestamp}")
        
    print("\nBlue Team Timestamps:")
    for caseNumber, timestamp in blueTimestamps.items():
        print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")

    print("\nRed Team Timestamps:")
    for caseNumber, timestamp in redTimestamps.items():
        print(f"Case Number: {caseNumber}, Timestamp: {timestamp}")
    processParticipantFramesToCsv(file_path, file_path, blueTimestamps, redTimestamps)
    
    df = pd.read_csv(file_path)
    filtered_df = df[df['participantId'].isin([3, 8])]
    filtered_df.to_csv(file_path, index=False)
    print('Filtered CSV file has been saved.')
    df = pd.read_csv(file_path)
    df['progressive_average_threat'] = df.groupby('participantId')['safetyStatus'].expanding().mean().reset_index(level=0, drop=True)
    df.to_csv(file_path, index=False)
    print(f"Modified file saved to {file_path}")
    df = pd.read_csv(file_path)
    df.tail(10)

        
        
demo_one_button_run(file_path)
df = pd.read_csv(file_path)
df.tail(10)
 

Team: 100, Tower Type: OUTER_TURRET, Lane: TOP_LANE, timestamp 1074561
Team: 200, Tower Type: OUTER_TURRET, Lane: BOT_LANE, timestamp 1162112
Team: 100, Tower Type: OUTER_TURRET, Lane: BOT_LANE, timestamp 1186441
Team: 100, Tower Type: OUTER_TURRET, Lane: MID_LANE, timestamp 1252821
Team: 200, Tower Type: OUTER_TURRET, Lane: TOP_LANE, timestamp 1349309
Team: 100, Tower Type: INNER_TURRET, Lane: MID_LANE, timestamp 1380698
Team: 200, Tower Type: INNER_TURRET, Lane: BOT_LANE, timestamp 1545648
Team: 100, Tower Type: INNER_TURRET, Lane: BOT_LANE, timestamp 1599129

Blue Team Timestamps:
Case Number: 1, Timestamp: 0
Case Number: 5, Timestamp: 1074561
Case Number: 11, Timestamp: 1186441
Case Number: 12, Timestamp: 1252821
Case Number: 19, Timestamp: 1380698
Case Number: 26, Timestamp: 1599129

Red Team Timestamps:
Case Number: 1, Timestamp: 0
Case Number: 7, Timestamp: 1162112
Case Number: 11, Timestamp: 1349309
Case Number: 16, Timestamp: 1545648
Calling function: blueCase1
Calling functio

,frameNumber,participantId,x,y,totalGold,totalDamageToChampions,safetyStatus,progressive_average_threat
48,25,3,5495,7048,6748,0,100,28.000000
49,25,8,7098,7501,11554,0,100,56.000000
50,26,3,4728,5062,6870,0,0,26.923077
51,26,8,12630,14011,12152,0,0,53.846154
52,27,3,605,898,7324,0,0,25.925926
53,27,8,1174,4800,12812,0,100,55.555556
54,28,3,2381,1654,7683,0,0,25.000000
55,28,8,7754,3422,13259,0,100,57.142857
56,29,3,5464,3380,7820,0,100,27.586207
57,29,8,578,2636,14166,0,100,58.620690
